Place for a picture

# Logistic Regression of a... (Phase Three Project)

## Business Problem/Question

What factors are most relevant to determining whether a private passenger vehicle crash in Chicago incurs property damage over $1,500? 

## EDA

In [123]:
# Importing packages
import numpy as np
import pandas as pd 
import math

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.dummy import DummyClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, plot_roc_curve
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

### Read in and create new csv

In [124]:
# Read in data

df_vehicles = pd.read_csv("Data\Traffic_Crashes_-_Vehicles_20231109.csv", parse_dates=["CRASH_DATE"], low_memory=False)
df_people = pd.read_csv("Data\Traffic_Crashes_-_People_20231109.csv", parse_dates=["CRASH_DATE"], low_memory=False)
df_crashes = pd.read_csv("Data\Traffic_Crashes_-_Crashes_20231109.csv", parse_dates=["CRASH_DATE"], low_memory=False)

In [125]:
# Filter out any data from before 2021

df_crashes = df_crashes[df_crashes["CRASH_DATE"].dt.year >= 2021]
df_people = df_people[df_people["CRASH_DATE"].dt.year >= 2021]
df_vehicles = df_vehicles[df_vehicles["CRASH_DATE"].dt.year >= 2021]

Using the data dictionaries to understand column names, we are dropping columns from each set that are not relevant to the business problem.

In [126]:
# Drop columns from Vehicles dataframe

df_vehicles.drop(columns = ['TOWED_I', 'FIRE_I', 'TOWED_BY', 'TOWED_TO', 'CMV_ID', 
                        'USDOT_NO', 'CCMC_NO', 'ILCC_NO', 'COMMERCIAL_SRC', 'GVWR', 
                        'CARRIER_NAME', 'CARRIER_STATE', 'CARRIER_CITY',
                        'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO', 'HAZMAT_PRESENT_I', 
                        'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO', 'MCS_REPORT_I',
                        'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I', 'MCS_VIO_CAUSE_CRASH_I', 
                        'IDOT_PERMIT_NO', 'WIDE_LOAD_I', 'TRAILER1_WIDTH', 'TRAILER2_WIDTH', 
                        'TRAILER1_LENGTH', 'TRAILER2_LENGTH', 'TOTAL_VEHICLE_LENGTH',
                        'AXLE_CNT', 'VEHICLE_CONFIG', 'CARGO_BODY_TYPE', 'LOAD_TYPE',
                        'HAZMAT_OUT_OF_SERVICE_I', 'MCS_OUT_OF_SERVICE_I', 'HAZMAT_CLASS'],
                         inplace=True)

In [127]:
# Drop columns from People dataframe

df_people.drop(columns = ['HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO'], inplace=True)

In [128]:
# Drop columns from Crashes dataframe

df_crashes.drop(columns = ['REPORT_TYPE', 'DATE_POLICE_NOTIFIED', 'PHOTOS_TAKEN_I',
                       'STATEMENTS_TAKEN_I', 'DOORING_I', 'INJURIES_TOTAL', 
                       'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 
                       'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT', 
                       'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN'], inplace=True)

Because we are most interested in vehicle damage, we are using the Vehicles dataframe as the main and merging the others into it.

In [129]:
# Merge People dataframe with Vehicles dataframe

df = df_vehicles.merge(df_people, how="left", on=["CRASH_RECORD_ID", "CRASH_DATE", "RD_NO", "VEHICLE_ID"])

In [130]:
# Merge Crashes dataframe with merged dataframe

df = pd.merge(df, df_crashes, how = 'inner', on = ['CRASH_RECORD_ID', "CRASH_DATE", "RD_NO"])

In [131]:
# Exporting new dataframe to csv for use throughout rest of notebook

df.to_csv("Data\chicago_traffic_accidents_2021_to_11-09-2023.csv")

### Working with a single merge dataset

In [132]:
# Can load merged dataframe without needing to go through above steps each time
df = pd.read_csv("Data\chicago_traffic_accidents_2021_to_11-09-2023.csv", low_memory=False)

We only want unique vehicle damage count, so we need to remove rows that represent passengers, as these will duplicate the vehicle damage. We also should remove any other rows that don't represent drivers. We can use the "PERSON_TYPE" column for this.

In [133]:
# Check values in Person_type column

df['PERSON_TYPE'].value_counts(normalize=True)

DRIVER                 0.780694
PASSENGER              0.197756
PEDESTRIAN             0.012899
BICYCLE                0.007730
NON-MOTOR VEHICLE      0.000760
NON-CONTACT VEHICLE    0.000162
Name: PERSON_TYPE, dtype: float64

In [134]:
# Remove all types of person except DRIVER

df = df[df['PERSON_TYPE'] == 'DRIVER']

In [135]:
# Sanity check

df['PERSON_TYPE'].value_counts(normalize=True)

DRIVER    1.0
Name: PERSON_TYPE, dtype: float64

### Missingness

Next we look at null values to try to determine which columns might need to be imputed or if the data is too incomplete to be useful. 

In [136]:
# First dropping columns that no longer have any data after removing all but DRIVER entries

df.dropna(axis=1, how="all", inplace=True)

In [137]:
# Looking at the total nulls left in remaining columns

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530319 entries, 0 to 766594
Data columns (total 89 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               530319 non-null  int64  
 1   CRASH_UNIT_ID            530319 non-null  int64  
 2   CRASH_RECORD_ID          530319 non-null  object 
 3   RD_NO                    522422 non-null  object 
 4   CRASH_DATE               530319 non-null  object 
 5   UNIT_NO                  530319 non-null  int64  
 6   UNIT_TYPE                530310 non-null  object 
 7   NUM_PASSENGERS           85079 non-null   float64
 8   VEHICLE_ID               530098 non-null  float64
 9   CMRC_VEH_I               10945 non-null   object 
 10  MAKE                     530098 non-null  object 
 11  MODEL                    530098 non-null  object 
 12  LIC_PLATE_STATE          469533 non-null  object 
 13  VEHICLE_YEAR             434488 non-null  float64
 14  VEHI

There are several columns that seem useful from the data dictionaries, but look almost entirely full of nulls. We do a value_counts for those columns to see what's in them.

In [138]:
# NUM_PASSENGERS

df["NUM_PASSENGERS"].value_counts(dropna=False)

NaN     445240
1.0      60610
2.0      15374
3.0       6037
4.0       2034
5.0        560
6.0        234
7.0         84
8.0         28
10.0        23
9.0         19
11.0        18
12.0        10
17.0         6
14.0         5
19.0         4
13.0         4
16.0         4
15.0         3
18.0         2
22.0         2
21.0         2
27.0         2
43.0         2
26.0         2
20.0         1
33.0         1
34.0         1
28.0         1
46.0         1
42.0         1
30.0         1
32.0         1
31.0         1
24.0         1
Name: NUM_PASSENGERS, dtype: int64

There is no 0 value, so we will cross check with the OCCUPANT_CNT column and impute 0 where OCCUPANT_CNT = 1. 

In [139]:
# CMRC_VEH_I

df["CMRC_VEH_I"].value_counts(dropna=False)

NaN    519374
Y        6602
N        4343
Name: CMRC_VEH_I, dtype: int64

This is a binary flag, but the missing values represent the overwhelming majority. We can drop the Y rows because they are commercial vehicles and do not fit the business problem, but the rest we will leave as we cannot make an assumption from such a small subset that the Y/N ratio is representative of the whole. 

In [140]:
# EXCEED_SPEED_LIMIT_I

df["EXCEED_SPEED_LIMIT_I"].value_counts(dropna=False)

NaN    530310
N           5
Y           4
Name: EXCEED_SPEED_LIMIT_I, dtype: int64

This is a binary flag, but the missing values represent the overwhelming majority. We cannot make an assumption from such a small subset that the Y/N ratio is representative of the whole. This does not seem to be a useful column for our initial model.

In [141]:
# AGE

df["AGE"].value_counts(dropna=False)

NaN      152847
28.0      10881
27.0      10830
29.0      10759
26.0      10716
          ...  
101.0         5
102.0         4
103.0         3
98.0          3
110.0         2
Name: AGE, Length: 106, dtype: int64

Missing values is a smaller percentage, so we can impute based on the average age. 

In [142]:
df["AGE"].mean()

39.979439534587996

In [143]:
# BAC_RESULT for BAC_RESULT VALUE

df["BAC_RESULT"].value_counts(dropna=False)

TEST NOT OFFERED                   522875
TEST REFUSED                         5377
TEST PERFORMED, RESULTS UNKNOWN      1197
TEST TAKEN                            870
Name: BAC_RESULT, dtype: int64

There are no nulls. We could create a binary flag feature as TESTED_FOR_BAC. 

In [144]:
# CELL_PHONE_USE

df["CELL_PHONE_USE"].value_counts(dropna=False)

NaN    530317
N           2
Name: CELL_PHONE_USE, dtype: int64

This is a binary flag, but the missing values represent the overwhelming majority. We cannot make an assumption from such a small subset that the Y/N ratio is representative of the whole. This does not seem to be a useful column for our initial model.

In [145]:
# LANE_CNT

df["LANE_CNT"].value_counts(dropna=False)

NaN    530248
2.0        25
4.0        21
1.0         7
3.0         7
6.0         4
5.0         4
0.0         2
8.0         1
Name: LANE_CNT, dtype: int64

The missing values represent the overwhelming majority. We cannot make an assumption from such a small subset that the ratio is representative of the whole. This does not seem to be a useful column for our initial model.

In [146]:
# INTERSECTION_RELATED_I

df["INTERSECTION_RELATED_I"].value_counts(dropna=False)

NaN    388222
Y      135705
N        6392
Name: INTERSECTION_RELATED_I, dtype: int64

This is a binary flag, but the missing values represent the majority. We may be able to impute values to the NaN because the Y/N values are a sizeable fraction of the whole, but we may want to leave this out of our initial model. 

In [147]:
# NOT_RIGHT_OF_WAY_I

df["NOT_RIGHT_OF_WAY_I"].value_counts(dropna=False)

NaN    511636
Y       16802
N        1881
Name: NOT_RIGHT_OF_WAY_I, dtype: int64

This is a binary flag, but the missing values represent the overwhelming majority. We cannot make an assumption from such a small subset that the Y/N ratio is representative of the whole. This does not seem to be a useful column for our initial model.

In [148]:
# HIT_AND_RUN_I

df["HIT_AND_RUN_I"].value_counts(dropna=False)

NaN    363846
Y      159186
N        7287
Name: HIT_AND_RUN_I, dtype: int64

This is a binary flag, but the missing values represent the majority. We may be able to impute values to the NaN because the Y/N values are a sizeable fraction of the whole, but we may want to leave this out of our initial model. We could assume that N is the default.

In [149]:
# WORK_ZONE_I 

df["WORK_ZONE_I"].value_counts(dropna=False)

NaN    527894
Y        1821
N         604
Name: WORK_ZONE_I, dtype: int64

This is a binary flag, but the missing values represent the overwhelming majority. We cannot make an assumption from such a small subset that the Y/N ratio is representative of the whole. This does not seem to be a useful column for our initial model.

## Feature Engineering

In [150]:
# Check the values in Damage column

df['DAMAGE'].value_counts()

OVER $1,500      364244
$501 - $1,500    121941
$500 OR LESS      44134
Name: DAMAGE, dtype: int64

In [151]:
# Create a new column to identify damage as > $1500 or <= $1500

damage_dict = {'OVER $1,500':'HIGH', '$501 - $1,500':'LOW', '$500 OR LESS':'LOW'}
df['DAMAGE_LEVEL'] =  df.loc[:, ('DAMAGE')].map(damage_dict).copy()

In [152]:
# Sanity check

df['DAMAGE_LEVEL'].value_counts()

HIGH    364244
LOW     166075
Name: DAMAGE_LEVEL, dtype: int64

In [153]:
# Create a new column to identify BAC_TEST as Y/N

bac_dict = {'TEST NOT OFFERED':0, 'TEST REFUSED':1, 'TEST PERFORMED, RESULTS UNKNOWN':1, 'TEST TAKEN':1}
df['BAC_TEST'] =  df.loc[:, ('BAC_RESULT')].map(bac_dict).copy()

In [154]:
# Sanity check

df["BAC_TEST"].value_counts()

0    522875
1      7444
Name: BAC_TEST, dtype: int64

## Decide Xs/y

In [155]:
df.describe()

,Unnamed: 0,CRASH_UNIT_ID,UNIT_NO,NUM_PASSENGERS,VEHICLE_ID,VEHICLE_YEAR,OCCUPANT_CNT,AGE,BAC_RESULT VALUE,POSTED_SPEED_LIMIT,LANE_CNT,STREET_NO,BEAT_OF_OCCURRENCE,NUM_UNITS,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,BAC_TEST
count,530319.000000,5.303190e+05,530319.000000,85079.000000,5.300980e+05,434488.000000,530098.000000,377472.000000,612.000000,530319.000000,71.000000,530319.000000,530319.000000,530319.000000,530319.000000,530319.00000,530319.000000,525675.000000,525675.000000,530319.000000
mean,390142.266426,1.361133e+06,1.446550,1.461971,1.293301e+06,2014.895571,1.234123,39.979440,0.175310,28.954733,3.014085,3743.570470,1238.961751,2.089508,13.342303,4.13705,6.422097,41.852162,-87.673383,0.014037
std,222517.264243,1.964298e+05,0.561206,1.024149,1.881972e+05,118.463723,0.674167,15.818965,0.105121,5.324429,1.553732,2851.585483,700.305196,0.485570,5.505126,1.97904,3.247378,0.366222,0.747839,0.117643
min,0.000000,1.017880e+06,1.000000,1.000000,9.647270e+05,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,111.000000,1.000000,0.000000,1.00000,1.000000,0.000000,-87.936193,0.000000
25%,195234.500000,1.191992e+06,1.000000,1.000000,1.130992e+06,2009.000000,1.000000,27.000000,0.130000,30.000000,2.000000,1329.000000,722.000000,2.000000,10.000000,2.00000,4.000000,41.780034,-87.722823,0.000000
50%,396971.000000,1.361150e+06,1.000000,1.000000,1.293444e+06,2014.000000,1.000000,37.000000,0.180000,30.000000,3.000000,3264.000000,1134.000000,2.000000,14.000000,4.00000,6.000000,41.871794,-87.675546,0.000000
75%,582789.500000,1.531866e+06,2.000000,2.000000,1.456725e+06,2018.000000,1.000000,51.000000,0.220000,30.000000,4.000000,5604.000000,1814.000000,2.000000,17.000000,6.00000,9.000000,41.924064,-87.633892,0.000000
max,766594.000000,1.699577e+06,12.000000,46.000000,1.617697e+06,9999.000000,47.000000,110.000000,1.000000,70.000000,8.000000,13799.000000,6100.000000,18.000000,23.000000,7.00000,12.000000,42.022780,0.000000,1.000000


In [4]:
X = df.drop("", axis=1)
y = df[""]

KeyError: "[''] not found in axis"

## Train/Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2024)

## Preprocessing Steps (SS, OHE, SI)

## Modeling (look at Coefficients, P-values)

## Evaluation OF/UF report Test

# Model 2

# Model 3